In [ ]:
import numpy as np
import glob
import os
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score,
    confusion_matrix, classification_report
)
from sklearn.model_selection import StratifiedKFold, cross_val_score, cross_validate
from google.colab import drive
from sklearn.model_selection import LeaveOneOut, GridSearchCV
import numpy as np
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.inspection import permutation_importance
from scipy.stats import ttest_ind
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
def apply_global_signal_regression(timeseries: np.ndarray) -> np.ndarray:
    global_signal = np.mean(timeseries, axis=1, keepdims=True)
    return timeseries - global_signal

def build_correlation_matrix(timeseries: np.ndarray) -> np.ndarray:
    """Compute correlation matrix while handling NaNs and checking constant regions."""
    timeseries = np.nan_to_num(timeseries, nan=0.0)
    variances = np.var(timeseries, axis=0)
    zero_var_regions = np.where(variances == 0)[0]
    if len(zero_var_regions) > 0:
        return None
    corr_matrix = np.corrcoef(timeseries, rowvar=False)
    return np.clip(np.nan_to_num(corr_matrix, nan=0.0, posinf=1.0, neginf=-1.0), -1.0, 1.0)

def flatten_upper_triangle(corr_matrix: np.ndarray) -> np.ndarray:
    """Extract the upper triangle (excluding diagonal) as a feature vector."""
    triu_indices = np.triu_indices(corr_matrix.shape[0], k=1)
    return corr_matrix[triu_indices]

def load_timeseries_data(ad_dir, cn_dir):
    ad_files = sorted(glob.glob(os.path.join(ad_dir, "*.txt")))
    cn_files = sorted(glob.glob(os.path.join(cn_dir, "*.txt")))

    X, y = [], []

    for fpath in ad_files:
        data_array = np.loadtxt(fpath)
        data_array = apply_global_signal_regression(data_array)
        corr_mat = build_correlation_matrix(data_array)

        if corr_mat is not None:
            X.append(flatten_upper_triangle(corr_mat))
            y.append(1)

    for fpath in cn_files:
        data_array = np.loadtxt(fpath)
        corr_mat = build_correlation_matrix(data_array)
        if corr_mat is not None:
            X.append(flatten_upper_triangle(corr_mat))
            y.append(0)

    return np.array(X), np.array(y)

def evaluate_model_with_cv(X, y, n_splits=5):
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)

    svm = SVC(kernel="rbf", C=1.0, gamma="scale")

    scoring = ['accuracy', 'precision', 'recall', 'f1']
    skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)
    scores = cross_validate(svm, X_scaled, y, cv=skf, scoring=scoring, return_train_score=False)

    results = {metric: (np.mean(scores[f'test_{metric}']), np.std(scores[f'test_{metric}']))
               for metric in scoring}

    sensitivities = []
    specificities = []

    for train_idx, test_idx in skf.split(X_scaled, y):
        svm.fit(X_scaled[train_idx], y[train_idx])
        y_pred = svm.predict(X_scaled[test_idx])
        cm = confusion_matrix(y[test_idx], y_pred)
        tn, fp, fn, tp = cm.ravel() if cm.shape == (2, 2) else (0, 0, 0, 0)
        sensitivity = tp / (tp + fn) if (tp + fn) > 0 else 0
        specificity = tn / (tn + fp) if (tn + fp) > 0 else 0
        sensitivities.append(sensitivity)
        specificities.append(specificity)

    results["Sensitivity"] = (np.mean(sensitivities), np.std(sensitivities))
    results["Specificity"] = (np.mean(specificities), np.std(specificities))

    return results

def evaluate_with_loocv(X, y):
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)

    svm = SVC(kernel="rbf", C=1.0, gamma="scale")
    loo = LeaveOneOut()

    y_true, y_pred = [], []
    for train_idx, test_idx in loo.split(X_scaled, y):
        svm.fit(X_scaled[train_idx], y[train_idx])
        pred = svm.predict(X_scaled[test_idx])
        y_true.append(y[test_idx][0])
        y_pred.append(pred[0])

    cm = confusion_matrix(y_true, y_pred)
    tn, fp, fn, tp = cm.ravel() if cm.shape == (2, 2) else (0, 0, 0, 0)
    sensitivity = tp / (tp + fn) if (tp + fn) > 0 else 0
    specificity = tn / (tn + fp) if (tn + fp) > 0 else 0

    results = {
        "accuracy": (accuracy_score(y_true, y_pred), 0.0),
        "precision": (precision_score(y_true, y_pred, zero_division=0), 0.0),
        "recall": (recall_score(y_true, y_pred, zero_division=0), 0.0),
        "f1": (f1_score(y_true, y_pred, zero_division=0), 0.0),
        "Sensitivity": (sensitivity, 0.0),
        "Specificity": (specificity, 0.0)
    }
    return results

def evaluate_with_nested_cv(X, y, outer_folds=5, inner_folds=3):
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)

    param_grid = {
        'C': [0.1, 1.0, 10.0],
        'gamma': ['scale', 0.1, 0.01]
    }

    outer_cv = StratifiedKFold(n_splits=outer_folds, shuffle=True, random_state=42)
    inner_cv = StratifiedKFold(n_splits=inner_folds, shuffle=True, random_state=42)

    accuracies, precisions, recalls, f1s, sensitivities, specificities = [], [], [], [], [], []

    for train_idx, test_idx in outer_cv.split(X_scaled, y):
        X_train, X_test = X_scaled[train_idx], X_scaled[test_idx]
        y_train, y_test = y[train_idx], y[test_idx]

        grid = GridSearchCV(SVC(kernel="rbf"), param_grid, cv=inner_cv, scoring='accuracy')
        grid.fit(X_train, y_train)

        best_model = grid.best_estimator_
        y_pred = best_model.predict(X_test)

        accuracies.append(accuracy_score(y_test, y_pred))
        precisions.append(precision_score(y_test, y_pred, zero_division=0))
        recalls.append(recall_score(y_test, y_pred, zero_division=0))
        f1s.append(f1_score(y_test, y_pred, zero_division=0))

        cm = confusion_matrix(y_test, y_pred)
        tn, fp, fn, tp = cm.ravel() if cm.shape == (2, 2) else (0, 0, 0, 0)
        sens = tp / (tp + fn) if (tp + fn) > 0 else 0
        spec = tn / (tn + fp) if (tn + fp) > 0 else 0
        sensitivities.append(sens)
        specificities.append(spec)

    results = {
        "accuracy": (np.mean(accuracies), np.std(accuracies)),
        "precision": (np.mean(precisions), np.std(precisions)),
        "recall": (np.mean(recalls), np.std(recalls)),
        "f1": (np.mean(f1s), np.std(f1s)),
        "Sensitivity": (np.mean(sensitivities), np.std(sensitivities)),
        "Specificity": (np.mean(specificities), np.std(specificities))
    }

    return results

def print_formatted_results(results_dict):
    print("\nFinal SVM Classification Results:")
    print("------------------------------------------------------")
    print(f"{'Metric':<20} {'Mean':<10} {'Std Dev'}")
    print("------------------------------------------------------")
    rename = {
        "accuracy": "Accuracy",
        "precision": "Precision (AD)",
        "recall": "Recall (AD)",
        "f1": "F1-score (AD)",
        "Sensitivity": "Sensitivity",
        "Specificity": "Specificity"
    }
    for metric, (mean, std) in results_dict.items():
        label = rename.get(metric, metric)
        print(f"{label:<20} {mean:.2f} ± {std:.2f}")
    print("------------------------------------------------------")

In [ ]:
## AAL90
AD_DIR = "/content/drive/MyDrive/FMRI/Exp1/Timeseries_AAL90/AD"
CN_DIR = "/content/drive/MyDrive/FMRI/Exp1/Timeseries_AAL90/CN"

X, y = load_timeseries_data(AD_DIR, CN_DIR)
results = evaluate_model_with_cv(X, y, n_splits=5)
print_formatted_results(results)


Final SVM Classification Results:
------------------------------------------------------
Metric               Mean       Std Dev
------------------------------------------------------
Accuracy             0.96 ± 0.08
Precision (AD)       0.94 ± 0.11
Recall (AD)          1.00 ± 0.00
F1-score (AD)        0.97 ± 0.07
Sensitivity          1.00 ± 0.00
Specificity          0.92 ± 0.16
------------------------------------------------------


In [ ]:
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

svm_linear = SVC(kernel="linear", C=1.0)
svm_linear.fit(X_scaled, y)

svm_weights = np.abs(svm_linear.coef_[0])
top_idx_svm = np.argsort(svm_weights)[-10:][::-1]

perm = permutation_importance(svm_linear, X_scaled, y, n_repeats=30, random_state=42)
perm_importance = perm.importances_mean
top_idx_perm = np.argsort(perm_importance)[-10:][::-1]

X_ad = X[np.array(y) == 1]
X_cn = X[np.array(y) == 0]
t_vals, p_vals = ttest_ind(X_ad, X_cn, axis=0)
top_idx_ttest = np.argsort(np.abs(t_vals))[-10:][::-1]

summary_df = pd.DataFrame({
    "SVM Weight Rank": top_idx_svm,
    "Permutation Rank": top_idx_perm,
    "T-test Rank": top_idx_ttest
})
print(summary_df)


   SVM Weight Rank  Permutation Rank  T-test Rank
0             1173                 0         2458
1             1218              4004         2499
2              484              4003         1604
3             1172              4002         2414
4             1651              4001         1602
5             2935                16         2402
6              732                17         2567
7             1210                18         3742
8             1174                19         2358
9             1159                20         3764


In [ ]:
def get_upper_triangle_pairs(n_regions=90):
    triu_idx = np.triu_indices(n_regions, k=1)
    return list(zip(triu_idx[0], triu_idx[1]))

region_pairs = get_upper_triangle_pairs()

for rank, idx in enumerate(top_idx_svm):
    i, j = region_pairs[idx]
    print(f"Rank {rank+1}: Feature {idx} → Region {i} ↔ Region {j}")


Rank 1: Feature 1173 → Region 14 ↔ Region 33
Rank 2: Feature 1218 → Region 14 ↔ Region 78
Rank 3: Feature 484 → Region 5 ↔ Region 55
Rank 4: Feature 1172 → Region 14 ↔ Region 32
Rank 5: Feature 1651 → Region 20 ↔ Region 82
Rank 6: Feature 2935 → Region 43 ↔ Region 55
Rank 7: Feature 732 → Region 8 ↔ Region 57
Rank 8: Feature 1210 → Region 14 ↔ Region 70
Rank 9: Feature 1174 → Region 14 ↔ Region 34
Rank 10: Feature 1159 → Region 14 ↔ Region 19
